## Create colabels

Use:
- This script will create a colabels file from seprate position and label files.

Requirements:
- A folder with position files containing bodyparts and objects.
- One or more folders with labels for those positions.


In [1]:
import pandas as pd
import os

In [2]:
base = r'c:\Users\dhers\Desktop\create_colabels'

position_folder = os.path.join(base, 'positions')
labeler_folders = ['labels_cande']
objects = ['obj_1', 'obj_2']

In [3]:
def create_colabels(position_folder, labelers, objects):
    """Creates colabels for a given folder of position files.

    Args:
        position_folder (str): Path to the folder containing position files.
        labelers (list): List of labelers names, each corresponding to a folder.
        objects (list): List of objects.
    """
    # Get list of position files
    position_files = [os.path.join(position_folder, f) for f in os.listdir(position_folder) if f.endswith('.csv')]
    
    # Initialize list to store concatenated data
    all_data = []
    
    for pos_file in position_files:
        # Load position data
        pos_df = pd.read_csv(pos_file)
        
        # Identify body part columns (excluding object positions)
        bodypart_cols = [col for col in pos_df.columns if not any(col.startswith(f'{obj}') for obj in objects)]
        bodyparts = pos_df[bodypart_cols]
        
        for obj in objects:
            # Extract object position columns
            obj_x = pos_df[f'{obj}_x']
            obj_y = pos_df[f'{obj}_y']
            
            # Load labeler data for this object
            labels = []
            for labeler in labelers:
                label_file = os.path.join(os.path.dirname(position_folder), labeler, os.path.basename(pos_file).replace('_position.csv', '_labels.csv'))
                label_df = pd.read_csv(label_file)
                labels.append(label_df[f'{obj}'])
            
            # Create a DataFrame with object positions, labels, and bodyparts
            obj_data = pd.DataFrame({'obj_x': obj_x, 'obj_y': obj_y})
            for i, label_col in enumerate(labels):
                obj_data[f'{labelers[i]}'] = label_col
            
            # Repeat bodypart positions for each object
            obj_data = pd.concat([bodyparts, obj_data], axis=1)
            
            all_data.append(obj_data)
    
    # Concatenate all objects' data vertically
    colabels_df = pd.concat(all_data, ignore_index=True)
    
    # Save to CSV
    output_file = os.path.join(os.path.dirname(position_folder), 'colabels.csv')
    colabels_df.to_csv(output_file, index=False)
    print(f'Colabels file saved as {output_file}')

In [4]:
create_colabels(position_folder, labeler_folders, objects)

Colabels file saved as c:\Users\dhers\Desktop\create_colabels\colabels.csv
